## 1. 환경 설정 및 모델 준비

### OpenAI 키 세팅

In [1]:
import os
import getpass

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("Colab Secrets에서 API 키를 성공적으로 불러왔습니다.")
except (ImportError, KeyError):
    try:
        api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
        os.environ["OPENAI_API_KEY"] = api_key
        print("API 키가 입력되었습니다.")
    except Exception as e:
        print(f"API 키를 설정하는 중 오류가 발생했습니다: {e}")
        exit()

API 키가 입력되었습니다.


### 문제 정의

다음 수학 문제를 세 가지 다른 프롬프트 엔지니어링 기법으로 해결해봅시다:

**문제**: 반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이를 구하세요.

정답: 5√3 ≈ 8.66 cm


## 2. 수학 계산 도구 정의

먼저 LLM이 사용할 수 있는 수학 계산 도구를 정의합니다.


In [8]:
from openai import OpenAI
import json
import math

# OpenAI 클라이언트 초기화
client = OpenAI()

# 수학 계산 함수 정의
def calculate(expression: str) -> float:
    """안전하게 수학 표현식을 계산합니다."""
    try:
        # 안전한 수학 함수들만 허용
        allowed_names = {
            k: v for k, v in math.__dict__.items() 
            if not k.startswith("__")
        }
        result = eval(expression, {"__builtins__": {}}, allowed_names)
        return result
    except Exception as e:
        return f"계산 오류: {str(e)}"

# OpenAI Function Calling을 위한 Tool 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "수학 표현식을 계산합니다. sqrt(), sin(), cos(), tan(), pi 등의 수학 함수를 사용할 수 있습니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "계산할 수학 표현식 (예: 'sqrt(3) * 5', 'sin(pi/6)')"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

# 문제 정의
MATH_PROBLEM = """다음 문제를 단계별로 풀어주세요:

문제: 반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이를 구하세요.

각 단계를 차근차근 분석하여 푸세요. 자세한 계산 과정을 보여주세요.
"""


## 3. Plan-and-Execute 방식

Plan-and-Execute는 문제를 해결하기 전에 먼저 전체 계획을 수립하고, 그 계획에 따라 순차적으로 실행하는 방식입니다.

### 특징
- **단계 1**: 문제 분석 및 해결 계획 수립
- **단계 2**: 계획에 따라 순차 실행
- **장점**: 명확한 구조, 체계적 접근
- **단점**: 계획 변경이 어려움, 유연성 부족


In [ ]:
def plan_and_execute(problem: str):
    """Plan-and-Execute: 먼저 전체 계획을 수립하고, 각 단계를 순차 실행"""
    
    # Step 1: 계획 수립
    planning_prompt = f"""다음 수학 문제를 해결하기 위한 단계별 계획을 세워주세요:

{problem}

계획은 다음 형식으로 작성하세요:
1. [단계 1 설명]
2. [단계 2 설명]
3. [단계 3 설명]
...

계획만 작성하고 실제 계산은 하지 마세요."""

    print("\n[계획 수립 단계]")
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": planning_prompt}],
        temperature=0
    )
    plan = response.choices[0].message.content
    print(plan)
    
    # Step 2: 계획 실행
    execution_prompt = f"""다음은 수학 문제를 해결하기 위한 계획입니다:

{plan}

이제 이 계획에 따라 문제를 단계별로 해결하세요. 
필요한 경우 calculate 함수를 사용하여 정확한 계산을 수행하세요.

원래 문제: {problem}"""

    print("\n[계획 실행 단계]")
    messages = [{"role": "user", "content": execution_prompt}]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        tools=tools,
        tool_choice="auto"
    )
    
    # Tool 호출 처리
    while response.choices[0].message.tool_calls:
        messages.append(response.choices[0].message)
        
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if function_name == "calculate":
                result = calculate(function_args["expression"])
                print(f"\n[계산]: {function_args['expression']} = {result}")
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0,
            tools=tools,
            tool_choice="auto"
        )
    
    final_answer = response.choices[0].message.content
    print(f"\n[최종 답변]\n{final_answer}")
    
    return final_answer

# 실행
plan_and_execute(MATH_PROBLEM)



[계획 수립 단계]
1. **문제 이해하기**: 반지름이 5cm인 원에 내접하는 정삼각형의 성질을 이해합니다. 정삼각형은 세 변의 길이가 같고, 각이 모두 60도인 삼각형입니다. 내접하는 정삼각형의 경우, 원의 중심과 정삼각형의 각 꼭짓점이 연결됩니다.

2. **정삼각형의 성질 활용하기**: 정삼각형의 한 변의 길이를 구하기 위해, 정삼각형의 중심에서 각 꼭짓점까지의 거리가 반지름과 같다는 점을 이용합니다. 이때, 정삼각형의 중심에서 한 변의 중점까지의 거리를 구하는 방법을 생각합니다.

3. **삼각형 구성하기**: 정삼각형의 한 변을 기준으로, 그 변의 중점을 정하고, 중심에서 중점까지의 수직선을 그립니다. 이 수직선은 정삼각형의 높이가 됩니다. 이 높이와 반지름을 이용하여 삼각형을 구성합니다.

4. **삼각형의 높이와 변의 길이 관계 찾기**: 정삼각형의 높이와 변의 길이 사이의 관계를 수학적으로 표현합니다. 정삼각형의 높이는 변의 길이의 비율을 통해 구할 수 있습니다. 이 비율을 이용하여 변의 길이를 구하는 공식을 세웁니다.

5. **계산하기**: 위에서 세운 공식을 바탕으로 반지름이 5cm인 경우의 변의 길이를 계산합니다. 필요한 경우, 삼각함수나 피타고라스의 정리를 활용하여 계산을 진행합니다.

6. **결과 확인하기**: 계산한 변의 길이가 올바른지 확인하기 위해, 정삼각형의 성질과 원의 반지름과의 관계를 다시 검토합니다. 필요한 경우, 다른 방법으로도 결과를 검증합니다.

[계획 실행 단계]

[최종 답변]
문제를 해결하기 위해 제시한 단계를 차근차근 진행하겠습니다.

### 1. 문제 이해하기
- 반지름이 5cm인 원에 내접하는 정삼각형을 고려합니다. 이 경우, 정삼각형의 꼭짓점들은 원의 경계에 위치하게 됩니다. 

### 2. 정삼각형의 성질 활용하기
- 정삼각형의 높이와 변의 길이 사이의 관계를 이해합니다. 정삼각형의 중심에서 각 꼭짓점까지의 거리는 반지름, 즉 5cm입니다.

### 3. 삼각형 구성하기
- 정삼각형의 한 변이 \(

'문제를 해결하기 위해 제시한 단계를 차근차근 진행하겠습니다.\n\n### 1. 문제 이해하기\n- 반지름이 5cm인 원에 내접하는 정삼각형을 고려합니다. 이 경우, 정삼각형의 꼭짓점들은 원의 경계에 위치하게 됩니다. \n\n### 2. 정삼각형의 성질 활용하기\n- 정삼각형의 높이와 변의 길이 사이의 관계를 이해합니다. 정삼각형의 중심에서 각 꼭짓점까지의 거리는 반지름, 즉 5cm입니다.\n\n### 3. 삼각형 구성하기\n- 정삼각형의 한 변이 \\(a\\)라고 했을 때, 정삼각형의 높이는 다음과 같이 구할 수 있습니다:\n  \\[\n  h = \\frac{\\sqrt{3}}{2} a\n  \\]\n- 여기서 \\(h\\)는 정삼각형의 높이입니다.\n\n### 4. 삼각형의 높이와 변의 길이 관계 찾기\n- 정삼각형의 중심에서 한 변의 중점까지의 거리와 높이에 대한 관계를 이해할 수 있습니다. 원의 반지름은 다음과 같습니다:\n  \\[\n  R = h + \\frac{a}{2}\n  \\]\n- 이때 \\(R\\)은 원의 반지름(5cm), \\(h\\)는 정삼각형의 높이입니다. 즉, 다음과 같은 관계가 성립됩니다:\n  \\[\n  5 = \\frac{\\sqrt{3}}{2} a + \\frac{a}{2}\n  \\]\n\n### 5. 계산하기\n위의 식을 정리하여 \\(a\\)를 구합니다:\n\\[\n5 = \\frac{\\sqrt{3}}{2} a + \\frac{1}{2} a\n\\]\n\\[\n5 = \\left(\\frac{\\sqrt{3}}{2} + \\frac{1}{2}\\right) a\n\\]\n\\[\n5 = \\left(\\frac{\\sqrt{3} + 1}{2}\\right) a\n\\]\n\\[\n10 = (\\sqrt{3} + 1)a\n\\]\n\\[\na = \\frac{10}{\\sqrt{3} + 1}\n\\]\n\n이와 같은 형태로 \\(a\\)를 남기고, 이를 계산합니다. 또한 분모의 유리화를 위해 \\(\\sqrt{3} 

## 4. ReAct (Reasoning + Acting) 방식

ReAct는 사고(Thought), 행동(Action), 관찰(Observation)을 반복하며 문제를 해결하는 방식입니다.

### 특징
- **Thought**: 현재 상황에 대한 사고
- **Action**: 취해야 할 행동 (도구 사용)
- **Observation**: 행동의 결과 관찰
- **장점**: 유연한 대응, 동적 문제 해결
- **단점**: 반복으로 인한 비효율성, 더 많은 API 호출


In [ ]:
def react_method(problem: str): # ①
    """ReAct: Thought(사고) - Action(행동) - Observation(관찰) 반복"""
    
    react_prompt = f"""당신은 수학 문제를 해결하는 에이전트입니다. 다음 형식으로 문제를 해결하세요:

Thought: [현재 상황에 대한 사고]
Action: [취해야 할 행동 - 계산이 필요하면 calculate 함수 사용]
Observation: [행동의 결과]

이 과정을 문제가 해결될 때까지 반복하세요.

문제: {problem}

단계별로 생각하고 필요한 계산을 수행하세요."""

    print("\n[ReAct 실행]")
    messages = [{"role": "user", "content": react_prompt}]
    
    max_iterations = 5
    iteration = 0
    
    while iteration < max_iterations: # ②
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0,
            tools=tools,
            tool_choice="auto"
        )
        
        assistant_message = response.choices[0].message
        print(f"\n{assistant_message.content}")
        
        if not assistant_message.tool_calls:
            # Tool 호출이 없으면 종료
            break
        
        messages.append(assistant_message)
        
        # ③ Tool 호출 처리
        for tool_call in assistant_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if function_name == "calculate":
                result = calculate(function_args["expression"])
                print(f"\n[Observation - 계산 결과]: {function_args['expression']} = {result}")
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
    
    # ④ 응답
    if iteration < max_iterations:
        final_answer = assistant_message.content
    else:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0,
        )
        final_answer = response.choices[0].message.content
        print(f"\n[최종 답변]\n{final_answer}")
    
    return final_answer

# 실행
react_method(MATH_PROBLEM)



[ReAct 실행]

--- Iteration 1 ---

Thought: 반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이를 구하기 위해, 먼저 정삼각형의 성질을 이해해야 합니다. 정삼각형의 내접원 반지름(R)과 변의 길이(a) 사이의 관계는 다음과 같습니다: \( R = \frac{a}{\sqrt{3}} \). 여기서 R은 내접원의 반지름입니다.

Action: 주어진 반지름 R = 5cm를 사용하여 변의 길이 a를 구하는 식을 세워보겠습니다. 이를 위해 식을 변형하겠습니다: \( a = R \cdot \sqrt{3} \).

Observation: 이제 \( a \)를 계산하기 위해 \( \sqrt{3} \)의 값을 구해야 합니다.

Action: calculate 함수를 사용하여 \( \sqrt{3} \)의 값을 계산합니다. 

```json
{
  "expression": "sqrt(3)"
}
```

[Observation - 계산 결과]: sqrt(3) = 1.7320508075688772

--- Iteration 2 ---

Observation: \( \sqrt{3} \)의 값은 약 1.732입니다. 이제 이 값을 사용하여 변의 길이 \( a \)를 계산할 수 있습니다.

Thought: 변의 길이 \( a \)를 구하기 위해 \( a = 5 \cdot \sqrt{3} \) 공식을 사용하겠습니다. 이제 \( \sqrt{3} \)의 값을 대입하여 계산하겠습니다.

Action: \( a = 5 \cdot 1.732 \)를 계산합니다.

```json
{
  "expression": "5 * 1.7320508075688772"
}
```

[Observation - 계산 결과]: 5 * 1.7320508075688772 = 8.660254037844386

--- Iteration 3 ---

Observation: 변의 길이 \( a \)는 약 8.66cm입니다. 

Thought: 이제 문제를 해결했습니다. 반지름이 5cm인 원에 내

'Observation: 변의 길이 \\( a \\)는 약 8.66cm입니다. \n\nThought: 이제 문제를 해결했습니다. 반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이는 약 8.66cm입니다.\n\nAction: 결과를 정리합니다.\n\nFinal Result: 반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이는 약 8.66cm입니다.'

## 5. Self-Reflection (자기 성찰) 방식

Self-Reflection은 자신의 답변을 비판적으로 평가하고 개선하는 방식입니다.

### 특징
- **단계 1**: 초기 답변 생성
- **단계 2**: Critic 역할로 답변 평가
- **단계 3**: 피드백을 반영하여 개선된 답변 생성
- **장점**: 높은 품질의 답변, 오류 자체 수정
- **단점**: 추가 API 호출, 시간 소요


In [12]:
def self_reflection(problem: str):
    """Self-Reflection: 전체 컨텍스트 유지"""
    
    # Step 1: 초기 답변 생성
    initial_prompt = f"""다음 수학 문제를 단계별로 해결하세요:

{problem}

각 단계를 명확히 설명하고, calculate 함수를 사용하여 정확한 계산을 수행하세요."""

    print("\n[초기 답변 생성]")
    messages = [{"role": "user", "content": initial_prompt}]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        tools=tools,
        tool_choice="auto"
    )
    
    # Tool 호출 처리
    while response.choices[0].message.tool_calls:
        assistant_msg = response.choices[0].message
        messages.append({
            "role": "assistant",
            "content": assistant_msg.content,
            "tool_calls": [
                {
                    "id": tc.id,
                    "type": tc.type,
                    "function": {
                        "name": tc.function.name,
                        "arguments": tc.function.arguments
                    }
                } for tc in assistant_msg.tool_calls
            ]
        })
        
        for tool_call in assistant_msg.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if function_name == "calculate":
                result = calculate(function_args["expression"])
                print(f"[계산]: {function_args['expression']} = {result}")
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0,
            tools=tools,
            tool_choice="auto"
        )
    
    initial_answer = response.choices[0].message.content
    messages.append({
        "role": "assistant",
        "content": initial_answer
    })
    print(f"\n초기 답변: {initial_answer}")
    
    # Step 2: 비판적 평가 (같은 대화 컨텍스트에서)
    critique_prompt = """이제 당신의 답변을 비판적으로 검토하세요:

1. **계산 검증**: 각 단계의 계산이 정확한가? 다시 확인하세요.
2. **논리 검증**: 문제 해결 과정에 빠진 단계나 오류가 있는가?
3. **검산**: 역산이나 다른 방법으로 답을 검증할 수 있는가?

예시:
- "2단계에서 15-3-5를 한번에 계산했는데, 순서대로 나눠서 해야 합니다"
- "7개를 받은 것을 빼먹었습니다"
- "검산: (15-3-5+7) = 14로 확인됩니다"

발견한 문제점을 구체적으로 나열하고, 문제가 없다면 "검토 완료: 오류 없음"이라고 하세요."""

    print("\n[비판적 평가]")
    messages.append({"role": "user", "content": critique_prompt})
    
    critique_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,  # 전체 컨텍스트 유지
        temperature=0
    )
    
    critique = critique_response.choices[0].message.content
    messages.append({"role": "assistant", "content": critique})
    print(f"\n{critique}")
    
    # Step 3: 개선 필요 여부 판단 및 실행
    if "오류 없음" in critique or "문제 없" in critique:
        print("\n[결과] 개선 불필요 - 초기 답변이 정확합니다.")
        return initial_answer
    
    improvement_prompt = """비평에서 지적한 문제점을 수정하여 최종 답변을 작성하세요.
필요하면 calculate 함수로 다시 계산하세요."""

    print("\n[개선된 답변 생성]")
    messages.append({"role": "user", "content": improvement_prompt})
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        tools=tools,
        tool_choice="auto"
    )
    
    # Tool 호출 처리
    while response.choices[0].message.tool_calls:
        assistant_msg = response.choices[0].message
        messages.append({
            "role": "assistant",
            "content": assistant_msg.content,
            "tool_calls": [
                {
                    "id": tc.id,
                    "type": tc.type,
                    "function": {
                        "name": tc.function.name,
                        "arguments": tc.function.arguments
                    }
                } for tc in assistant_msg.tool_calls
            ]
        })
        
        for tool_call in assistant_msg.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if function_name == "calculate":
                result = calculate(function_args["expression"])
                print(f"[재계산]: {function_args['expression']} = {result}")
                
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(result)
                })
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0,
            tools=tools,
            tool_choice="auto"
        )
    
    improved_answer = response.choices[0].message.content
    print(f"\n[최종 답변]\n{improved_answer}")
    
    return improved_answer

# 실행
self_reflection(MATH_PROBLEM)



[초기 답변 생성]
[계산]: 5 * sqrt(3) = 8.660254037844386

초기 답변: ### 5단계: 결과 확인하기
계산 결과, 정삼각형의 한 변의 길이 \( a \)는 약 8.66 cm입니다.

### 최종 결과
반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이는 약 **8.66 cm**입니다.

[비판적 평가]

### 1. 계산 검증
각 단계의 계산을 다시 확인해보겠습니다.

- **2단계**에서 \( R = 5 \)를 대입하여 \( a = 5 \cdot \sqrt{3} \)로 정리한 것은 정확합니다.
- **4단계**에서 \( 5 \cdot \sqrt{3} \)의 값을 계산한 결과는 약 8.66 cm로, 이는 정확합니다.

### 2. 논리 검증
문제 해결 과정에서 빠진 단계나 오류는 없습니다. 정삼각형의 성질과 내접원의 반지름과의 관계를 올바르게 사용하였고, 각 단계를 논리적으로 연결하여 문제를 해결했습니다.

### 3. 검산
검산을 위해 다른 방법으로도 확인해보겠습니다. 정삼각형의 내접원의 반지름과 변의 길이의 관계를 다시 확인해보면:

\[
R = \frac{a}{\sqrt{3}} \implies a = R \cdot \sqrt{3}
\]

여기서 \( R = 5 \)를 대입하면:

\[
a = 5 \cdot \sqrt{3} \approx 8.66
\]

또한, 정삼각형의 높이를 구하고, 이를 통해 변의 길이를 구하는 방법도 있습니다. 정삼각형의 높이는 다음과 같이 구할 수 있습니다:

\[
h = \frac{\sqrt{3}}{2} a
\]

내접원의 반지름 R은 높이와 관련이 있으므로:

\[
R = \frac{h}{3} = \frac{\frac{\sqrt{3}}{2} a}{3} = \frac{\sqrt{3}}{6} a
\]

이 식을 통해도 \( a \)를 구할 수 있습니다. \( R = 5 \)를 대입하면:

\[
5 = \frac{\sqrt{3}}{6} a \implies a = 5 \cdot \frac{6}{\s

'### 5단계: 결과 확인하기\n계산 결과, 정삼각형의 한 변의 길이 \\( a \\)는 약 8.66 cm입니다.\n\n### 최종 결과\n반지름이 5cm인 원에 내접하는 정삼각형의 한 변의 길이는 약 **8.66 cm**입니다.'